In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE GeneralizedNewtypeDeriving #-}

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings #-}
Why not:
Line 2: Unused LANGUAGE pragma
Found:
{-# LANGUAGE DeriveGeneric #-}
Why not:
Line 3: Unused LANGUAGE pragma
Found:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}
Why not:

In [2]:
import Data.Aeson
import Data.Aeson.Types
import Control.Applicative (Alternative(..))
import qualified Data.ByteString.Lazy as BSL

In [3]:
data Fruit = Apple Int | Orange String deriving (Eq, Show)

instance FromJSON Fruit where
    parseJSON o@(Object v) = do
        appleM <- v .:? "apple"
        orangeM <- v .:? "orange"
        case (appleM, orangeM) of
            (Just apple, _) -> pure $ Apple apple
            (_, Just orange) -> pure $ Orange orange 
            _ -> fail "neither apple nor orange" o
    parseJSON invalid = typeMismatch "Fruit" invalid

In [4]:
newtype Carrot = Carrot [Int] deriving (Eq, Show, FromJSON, ToJSON)

newtype CarrotContainer = CarrotContainer Carrot deriving (Eq, Show)

instance FromJSON CarrotContainer where
    parseJSON = withObject "CarrotContainer" $ \o ->
        CarrotContainer <$> o .: "carrot"

In [5]:
newtype FruitOrVeg = FruitOrVeg (Either Fruit Carrot) deriving (Eq, Show)

instance FromJSON FruitOrVeg where
    parseJSON = withObject "FruitOrVeg" $ \o ->
        FruitOrVeg <$> ((Left <$> parseJSON (Object o)) <|> (Right <$> o .: "carrot"))

In [6]:
orange <- BSL.readFile "./orange.json"
carrot <- BSL.readFile "./carrot.json"

In [7]:
eitherDecode orange :: Either String FruitOrVeg

Right (FruitOrVeg (Left (Orange "orange")))

In [8]:
eitherDecode carrot :: Either String CarrotContainer

Right (CarrotContainer (Carrot [1,2,3]))

In [9]:
eitherDecode carrot :: Either String FruitOrVeg

: 

In [10]:
newtype FixedFruit = FixedFruit Fruit deriving (Eq, Show)

instance FromJSON FixedFruit where
    parseJSON o@(Object v) = do
        appleM <- v .:? "apple"
        orangeM <- v .:? "orange"
        case (appleM, orangeM) of
            (Just apple, _) -> pure . FixedFruit $ Apple apple
            (_, Just orange) -> pure . FixedFruit $ Orange orange 
            _ -> fail "neither apple nor orange"

In [11]:
newtype FixedFruitOrVeg = FixedFruitOrVeg (Either FixedFruit Carrot) deriving (Eq, Show)

instance FromJSON FixedFruitOrVeg where
    parseJSON = withObject "FruitOrVeg" $ \o ->
        FixedFruitOrVeg <$> ((Left <$> parseJSON (Object o)) <|> (Right <$> o .: "carrot"))

In [12]:
eitherDecode carrot :: Either String FixedFruitOrVeg

Right (FixedFruitOrVeg (Right (Carrot [1,2,3])))